# Create and Train Model

## Create Paths

In [5]:
import os
from string import ascii_uppercase

custom_model_name = 'ASL_prototype_labeling'
pretrained_model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
pretrained_model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
tf_record_script_name = 'generate_tfrecord.py'
label_map_name = 'ASL_label_map.pbtxt'

paths = {
    'workspace': os.path.join('ASL_AI'),
    'scripts': os.path.join('tensorflow', 'scripts'),
    'apimodel_path': os.path.join('tensorflow', 'models'),
    'annotation_path': os.path.join('ASL_AI', 'annotations'),
    'image_path': os.path.join('Images', 'Captured_ASL'),
    'model_path': os.path.join('ASL_AI', 'models'),
    'pretrained_model_path': os.path.join('tensorflow', 'workspace', 'pre-trained-models'),
    'checkpoint_path': os.path.join('ASL_AI', 'models', custom_model_name),
    'output_path': os.path.join('ASL_AI', 'models', custom_model_name, 'export'),
    'protoc_path': os.path.join('tensorflow', 'protoc')
}

files = {
    'pipeline_config': os.path.join('ASL_AI', 'models', custom_model_name, 'pipeline.config'),
    'tf_record_script': os.path.join(paths['scripts'], tf_record_script_name),
    'label_map': os.path.join(paths['annotation_path'], label_map_name)
}

for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## Download model

In [14]:
if os.name == 'posix':
    !wget {pretrained_model_url}
    !mv {pretrained_model_name+'.tar.gz'} {paths['pretrained_model_path']}
    !cd {paths['pretrained_model_path']} && tar -zxvf {pretrained_model_name+'.tar.gz'}
if os.name == 'nt':
    import wget
    wget.download(pretrained_model_url)
    !move {pretrained_model_name+'.tar.gz'} {paths['pretrained_model_path']}
    !cd {paths['pretrained_model_path']} && tar -zxvf {pretrained_model_name+'.tar.gz'}

100% [........................................................................] 20518283 / 20518283        1 file(s) moved.


x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [6]:
labels = []
counter = 1
for char in ascii_uppercase:
    labels.append({'name': 'ASL_'+char, 'id': counter})
    counter += 1
print(labels)

#labels = [
#    {'name': 'ASL_A', 'id': 1},
#    {'name': 'ASL_B', 'id': 2}
#]

with open(files['label_map'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write(f'\tname:\'{label["name"]}\'\n')
        f.write(f'\tid:{label["id"]}\n')
        f.write('}\n')

[{'name': 'ASL_A', 'id': 1}, {'name': 'ASL_B', 'id': 2}, {'name': 'ASL_C', 'id': 3}, {'name': 'ASL_D', 'id': 4}, {'name': 'ASL_E', 'id': 5}, {'name': 'ASL_F', 'id': 6}, {'name': 'ASL_G', 'id': 7}, {'name': 'ASL_H', 'id': 8}, {'name': 'ASL_I', 'id': 9}, {'name': 'ASL_J', 'id': 10}, {'name': 'ASL_K', 'id': 11}, {'name': 'ASL_L', 'id': 12}, {'name': 'ASL_M', 'id': 13}, {'name': 'ASL_N', 'id': 14}, {'name': 'ASL_O', 'id': 15}, {'name': 'ASL_P', 'id': 16}, {'name': 'ASL_Q', 'id': 17}, {'name': 'ASL_R', 'id': 18}, {'name': 'ASL_S', 'id': 19}, {'name': 'ASL_T', 'id': 20}, {'name': 'ASL_U', 'id': 21}, {'name': 'ASL_V', 'id': 22}, {'name': 'ASL_W', 'id': 23}, {'name': 'ASL_X', 'id': 24}, {'name': 'ASL_Y', 'id': 25}, {'name': 'ASL_Z', 'id': 26}]


In [7]:
if not os.path.exists(files['tf_record_script']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

!python C:\Users\redth\Desktop\ODT\tensorflow\scripts\generate_tfrecord.py -x C:\Users\redth\Desktop\ODT\Images\Captured_ASL\train -l C:\Users\redth\Desktop\ODT\ASL_AI\annotations\ASL_label_map.pbtxt -o C:\Users\redth\Desktop\ODT\ASL_AI\annotations\train.record 
!python C:\Users\redth\Desktop\ODT\tensorflow\scripts\generate_tfrecord.py -x C:\Users\redth\Desktop\ODT\Images\Captured_ASL\test -l C:\Users\redth\Desktop\ODT\ASL_AI\annotations\ASL_label_map.pbtxt -o C:\Users\redth\Desktop\ODT\ASL_AI\annotations\test.record

Successfully created the TFRecord file: C:\Users\redth\Desktop\ODT\ASL_AI\annotations\train.record
Successfully created the TFRecord file: C:\Users\redth\Desktop\ODT\ASL_AI\annotations\test.record


## Update model configuration

In [8]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

if os.name == 'posix':
    !cp {os.path.join(paths['pretrained_model_path'], pretrained_model_name, 'pipeline.config')} {os.path.join(paths['checkpoint_path'])}
if os.name == 'nt':
    !copy {os.path.join(paths['pretrained_model_path'], pretrained_model_name, 'pipeline.config')} {os.path.join(paths['checkpoint_path'])}
    
config = config_util.get_configs_from_pipeline_file(files['pipeline_config'])

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['pipeline_config'], 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)
    
pipeline_config.model.ssd.num_classes = 26
pipeline_config.train_config.batch_size = 16
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['pretrained_model_path'], pretrained_model_name, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = files['label_map']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['annotation_path'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['label_map']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['annotation_path'], 'test.record')]

pipeline_config.eval_config.num_visualizations = 100

config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['pipeline_config'], 'wb') as f:
    f.write(config_text)

Num GPUs Available:  1
        1 file(s) copied.


## Train Model

command: python tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=ASL_AI\models\ASL_prototype_labeling --pipeline_config_path=ASL_AI\models\ASL_prototype_labeling\pipeline.config --num_train_steps=2000

In [9]:
training_script = os.path.join(paths['apimodel_path'], 'research', 'object_detection', 'model_main_tf2.py')
command = f"python {training_script} --model_dir={paths['checkpoint_path']} --pipeline_config_path={files['pipeline_config']} --num_train_steps=3000"
print(command)

python tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=ASL_AI\models\ASL_prototype_labeling --pipeline_config_path=ASL_AI\models\ASL_prototype_labeling\pipeline.config --num_train_steps=3000


## Eval Model

command: python tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=ASL_AI\models\ASL_prototype_labeling --pipeline_config_path=ASL_AI\models\ASL_prototype_labeling\pipeline.config --checkpoint_dir=ASL_AI\models\ASL_prototype_labeling

In [12]:
command = f"python {training_script} --model_dir={paths['checkpoint_path']} --pipeline_config_path={files['pipeline_config']} --checkpoint_dir={paths['checkpoint_path']}"
print(command)

python tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=ASL_AI\models\ASL_prototype_labeling --pipeline_config_path=ASL_AI\models\ASL_prototype_labeling\pipeline.config --checkpoint_dir=ASL_AI\models\ASL_prototype_labeling


## Tensorboard

- Train: cd ASL_AI/models/ASL_prototype_labeling/train && tensorboard --logdir .
- Eval: cd ASL_AI/models/ASL_prototype_labeling/eval && tensorboard --logdir .